In [1]:
import React from 'react';
import Highcharts from 'highcharts/highstock';
import HighchartsReact from 'highcharts-react-official';

const StockChart = ({ data }) => {
  const options = {
    title: {
      text: 'Stock Price'
    },
    xAxis: {
      type: 'datetime'
    },
    yAxis: {
      title: {
        text: 'Price'
      }
    },
    series: [{
      name: 'Stock Price',
      data: data,
      tooltip: {
        valueDecimals: 2
      }
    }]
  };

  return (
    <HighchartsReact
      highcharts={Highcharts}
      constructorType={'stockChart'}
      options={options}
    />
  );
};

export default StockChart;

SyntaxError: invalid syntax (2490503976.py, line 1)

In [ ]:
 if (isChatActive) {
    return (
      <div className={styles.chatPage}>
        <div className={styles.chatHeader}>
          <div className={styles.chatHeaderTitle}>
            <div className={styles.iconWrapper}>
              <FluentIcons.Sparkle24Filled />
            </div>
            <Text className={styles.chatHeaderText}>{userConfig.buttons[0].label}</Text>
          </div>
          <Text className={styles.chatHeaderDescription}>{userConfig.buttons[0].description}</Text>
        </div>
        <div className={styles.chatMessages}>
          {chatMessages.map((message, index) => (
            <React.Fragment key={index}>
              <div
                className={`${styles.chatMessage} ${
                  message.role === "user" ? styles.userMessage : styles.assistantMessage
                }`}
              >
                {message.role === "user" ? (
                  message.content
                ) : (
                  <div dangerouslySetInnerHTML={{ __html: message.content }} />
                )}
              </div>
              {message.role === "user" && index === 0 && isFirstQuery && (
                <div className={styles.chartContainer}>
                  <StockChart data={stockData} />
                </div>
              )}
              {message.subgraphResults && (
                <div className={styles.subgraphMessage}>
                  <SubgraphResults results={message.subgraphResults} />
                </div>
              )}
            </React.Fragment>
          ))}
        </div>

In [ ]:
import React, { useState, useEffect, useCallback } from 'react';
import {
  Button,
  Text,
  makeStyles,
  tokens,
} from "@fluentui/react-components";
import * as FluentIcons from "@fluentui/react-icons";
import SubgraphResults from './SubgraphResults';
import StockChart from './StockChart';
import DynamicProgressBar from './DynamicProgressBar';

const useStyles = makeStyles({
  // ... (keep existing styles)
  chartContainer: {
    height: '400px',
    marginBottom: '20px',
  },
});

const EmailGenerator = () => {
  const styles = useStyles();
  const [chatMessages, setChatMessages] = useState([]);
  const [chatInput, setChatInput] = useState("");
  const [isProcessing, setIsProcessing] = useState(false);
  const [currentStep, setCurrentStep] = useState("");
  const [isChatActive, setIsChatActive] = useState(false);
  const [isFirstQuery, setIsFirstQuery] = useState(true);
  const [stockData, setStockData] = useState([]);
  const finalResponseRef = useRef(null);

  useEffect(() => {
    // Set up your stockData here
    const hardcodedStockData = [
      [Date.UTC(2023, 0, 1), 100],
      [Date.UTC(2023, 1, 1), 110],
      [Date.UTC(2023, 2, 1), 105],
      [Date.UTC(2023, 3, 1), 120],
      [Date.UTC(2023, 4, 1), 115],
      // ... add more data points as needed
    ];
    setStockData(hardcodedStockData);
  }, []);

  const handleUpdate = useCallback((data) => {
    if (data.type === 'step') {
      setCurrentStep(data.content);
    } else if (data.type === 'subgraph_results') {
      setChatMessages(prevMessages => {
        const lastMessageIndex = prevMessages.length - 1;
        const updatedMessages = [...prevMessages];
        updatedMessages[lastMessageIndex] = {
          ...updatedMessages[lastMessageIndex],
          subgraphResults: data.content,
        };
        return updatedMessages;
      });
      console.log("Received subgraph results:", data.content);
    } else if (data.type === 'final') {
      finalResponseRef.current = data.content;
      setChatMessages(prevMessages => {
        // Check if the last message is already the final response
        const lastMessage = prevMessages[prevMessages.length - 1];
        if (lastMessage && lastMessage.role === "assistant" && lastMessage.content === wrapInHtml(data.content)) {
          return prevMessages; // No need to add the final response again
        }
        return [
          ...prevMessages,
          { role: "assistant", content: wrapInHtml(data.content) }
        ];
      });
      console.log("Received final response:", data.content);
    }
  }, []);

  const handleChatSubmit = (e) => {
    e.preventDefault();
    if (chatInput.trim() && !isProcessing) {
      console.log("Submitting chat input:", chatInput);
      handleAction(chatInput);
      setChatInput("");
      if (isFirstQuery) {
        setIsFirstQuery(false);
      }
    }
  };

  // ... (keep other existing functions)

  if (isChatActive) {
    return (
      <div className={styles.chatPage}>
        <div className={styles.chatHeader}>
          <div className={styles.chatHeaderTitle}>
            <div className={styles.iconWrapper}>
              <FluentIcons.Sparkle24Filled />
            </div>
            <Text className={styles.chatHeaderText}>{userConfig.buttons[0].label}</Text>
          </div>
          <Text className={styles.chatHeaderDescription}>{userConfig.buttons[0].description}</Text>
        </div>
        <div className={styles.chatMessages}>
          {chatMessages.map((message, index) => (
            <React.Fragment key={index}>
              <div
                className={`${styles.chatMessage} ${
                  message.role === "user" ? styles.userMessage : styles.assistantMessage
                }`}
              >
                {message.role === "user" ? (
                  message.content
                ) : (
                  <div dangerouslySetInnerHTML={{ __html: message.content }} />
                )}
              </div>
              {message.role === "user" && index === 0 && (
                <div className={styles.chartContainer}>
                  <StockChart data={stockData} />
                </div>
              )}
              {message.subgraphResults && (
                <SubgraphResults results={message.subgraphResults} />
              )}
            </React.Fragment>
          ))}
        </div>
        {isProcessing && <DynamicProgressBar currentStep={currentStep} />}
        <form onSubmit={handleChatSubmit} className={styles.chatInput}>
          <input
            type="text"
            value={chatInput}
            onChange={(e) => setChatInput(e.target.value)}
            placeholder="Type your message..."
            className={styles.chatInputField}
            disabled={isProcessing}
          />
          <Button type="submit" disabled={isProcessing || !chatInput.trim()}>
            Send
          </Button>
        </form>
      </div>
    );
  }

  // ... (rest of the component)
};

export default EmailGenerator;